# Traing Model Through Transfer Learning (VGG 16)

---

## Description
This script will train the model using Transfer Learning (VGG 16) to do binary classification for classes:
* Damaged
* Preserved

This should provide us a good starting point and set standards for base accuracy.

## About Dataset
164 images will be the part of training set and 20 images for the test set. This dataset is created manually to improve model's accuracy. Following is hierarchy for dataset folder.

```
project   
│
└───dataset
│   │
│   └───test_set
│   │   │
│   │   └───damaged (10 images)
│   │   │   83.jpg
│   │   │   84.jpg
│   │   │   ...
│   │   │
│   │   └───preserved (10 images)
│   │       83.jpg
│   │       84.jpg
│   │       ...
│   │  
│   └───training_set
│       │
│       └───damaged (82 images)
│       │   1.jpg
│       │   2.jpg
│       │   ...
│       │
│       └───preserved (82 images)
│           1.jpg
│           2.jpg
│           ...
│     
```



## Dataset Source
Dataset can be downloaded from Google Drive or ask Muhammad Ali for dataset related to this specific version.


## Training Results
The model showed __0.658% Loss__ & __0.899% Accuracy__ on validation data during training with following parameters
* batch_size = 2
* epochs = 50
* train_samples = 164
* validation_samples = 20


### NOTE
This model is trained using the images of __EARTHQUAKE__ only.



In [ ]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'dataset/training_set'
validation_data_dir = 'dataset/test_set'

batch_size = 2
epochs = 50
train_samples = 164
validation_samples = 20

C:\Users\Javapocalypse\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_vgg = applications.VGG16(include_top=False)



In [3]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)


Found 164 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [4]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, train_samples // batch_size)
np.save(open('bottleneck_features_train.npy', 'wb+'), bottleneck_features_train)

In [5]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, validation_samples // batch_size)
np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

In [6]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (train_samples // 2) + [1] * (train_samples // 2))

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (validation_samples // 2) + [1] * (validation_samples // 2))

In [7]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:

model_top.fit(train_data, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_data, validation_labels))

Train on 164 samples, validate on 20 samples
Epoch 1/50
164/164 [==============================] - 2s 13ms/step - loss: 1.2179 - acc: 0.6768 - val_loss: 0.9009 - val_acc: 0.6500
Epoch 2/50
164/164 [==============================] - 0s 3ms/step - loss: 0.6616 - acc: 0.7866 - val_loss: 0.3203 - val_acc: 0.8500
Epoch 3/50
164/164 [==============================] - 0s 3ms/step - loss: 0.3597 - acc: 0.8415 - val_loss: 0.3235 - val_acc: 0.8500
Epoch 4/50
164/164 [==============================] - 0s 3ms/step - loss: 0.4704 - acc: 0.8598 - val_loss: 0.6956 - val_acc: 0.7000
Epoch 5/50
164/164 [==============================] - 0s 3ms/step - loss: 0.2444 - acc: 0.9573 - val_loss: 0.4566 - val_acc: 0.8500
Epoch 6/50
164/164 [==============================] - 0s 3ms/step - loss: 0.2303 - acc: 0.9085 - val_loss: 0.4377 - val_acc: 0.8500
Epoch 7/50
164/164 [==============================] - 0s 3ms/step - loss: 0.2095 - acc: 0.9024 - val_loss: 0.5584 - val_acc: 0.7000
Epoch 8/50
164/164 [==========

In [9]:
model_top.save_weights('bottleneck_50_epochs.h5')

In [10]:
model_top.evaluate(validation_data, validation_labels)

20/20 [==============================] - 0s 226us/step


[0.6583938598632812, 0.8999999761581421]